
# Lab 3.02: Statistical Modeling and Model Validation


---

## Objective
The goal of this lab is to guide you through the modeling workflow to produce the best model you can. In this lesson, you will follow all best practices when slicing your data and validating your model. 

## Imports

In [6]:
# Import everything you need here.
# You may want to return to this cell to import more things later in the lab.
# DO NOT COPY AND PASTE FROM OUR CLASS SLIDES!
# Muscle memory is important!
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, mean_squared_error
import math
from scipy.stats import norm
import statsmodels.api as sm

## Read Data
The `citibike` dataset consists of Citi Bike ridership data for over 224,000 rides in February 2014.

In [7]:
# Read in the citibike data in the data folder in this repository.
citibike = pd.read_csv("data/citibike_feb2014.csv")
citibike.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,382,2014-02-01 00:00:00,2014-02-01 00:06:22,294,Washington Square E,40.730494,-73.995721,265,Stanton St & Chrystie St,40.722293,-73.991475,21101,Subscriber,1991,1
1,372,2014-02-01 00:00:03,2014-02-01 00:06:15,285,Broadway & E 14 St,40.734546,-73.990741,439,E 4 St & 2 Ave,40.726281,-73.989780,15456,Subscriber,1979,2
2,591,2014-02-01 00:00:09,2014-02-01 00:10:00,247,Perry St & Bleecker St,40.735354,-74.004831,251,Mott St & Prince St,40.723180,-73.994800,16281,Subscriber,1948,2
3,583,2014-02-01 00:00:32,2014-02-01 00:10:15,357,E 11 St & Broadway,40.732618,-73.991580,284,Greenwich Ave & 8 Ave,40.739017,-74.002638,17400,Subscriber,1981,1
4,223,2014-02-01 00:00:41,2014-02-01 00:04:24,401,Allen St & Rivington St,40.720196,-73.989978,439,E 4 St & 2 Ave,40.726281,-73.989780,19341,Subscriber,1990,1


## Explore the data
Use this space to familiarize yourself with the data.

Convince yourself there are no issues with the data. If you find any issues, clean them here.

In [8]:
citibike["starttime"] = pd.to_datetime(citibike["starttime"])
citibike["stoptime"] = pd.to_datetime(citibike["stoptime"])

In [9]:
citibike.isnull().values.any()

False

In [10]:
citibike[citibike["usertype"] == "Customer"]

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
31,664,2014-02-01 00:08:47,2014-02-01 00:19:51,237,E 11 St & 2 Ave,40.730473,-73.986724,349,Rivington St & Ridge St,40.718502,-73.983299,17540,Customer,\N,0
55,836,2014-02-01 00:16:10,2014-02-01 00:30:06,488,W 39 St & 9 Ave,40.756458,-73.993722,297,E 15 St & 3 Ave,40.734232,-73.986923,16731,Customer,\N,0
222,1277,2014-02-01 01:17:50,2014-02-01 01:39:07,469,Broadway & W 53 St,40.763441,-73.982681,336,Sullivan St & Washington Sq,40.730477,-73.999061,20728,Customer,\N,0
266,29906,2014-02-01 01:44:59,2014-02-01 10:03:25,294,Washington Square E,40.730494,-73.995721,368,Carmine St & 6 Ave,40.730386,-74.002150,18944,Customer,\N,0
293,2625,2014-02-01 01:56:32,2014-02-01 02:40:17,395,Bond St & Schermerhorn St,40.688070,-73.984106,395,Bond St & Schermerhorn St,40.688070,-73.984106,19782,Customer,\N,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224385,280,2014-02-28 21:42:34,2014-02-28 21:47:14,161,LaGuardia Pl & W 3 St,40.729170,-73.998102,382,University Pl & E 14 St,40.734927,-73.992005,15529,Customer,\N,0
224438,1106,2014-02-28 22:00:46,2014-02-28 22:19:12,423,W 54 St & 9 Ave,40.765849,-73.986905,385,E 55 St & 2 Ave,40.757973,-73.966033,18720,Customer,\N,0
224525,864,2014-02-28 22:29:52,2014-02-28 22:44:16,385,E 55 St & 2 Ave,40.757973,-73.966033,441,E 52 St & 2 Ave,40.756014,-73.967416,21399,Customer,\N,0
224536,683,2014-02-28 22:32:55,2014-02-28 22:44:18,385,E 55 St & 2 Ave,40.757973,-73.966033,441,E 52 St & 2 Ave,40.756014,-73.967416,17516,Customer,\N,0


In [11]:
citibike["usertype"].describe()

count         224736
unique             2
top       Subscriber
freq          218019
Name: usertype, dtype: object

## Is average trip duration different by gender?

Conduct a hypothesis test that checks whether or not the average trip duration is different for `gender=1` and `gender=2`. Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly!

In [12]:
gender_1 = citibike[citibike["gender"] == 1]["tripduration"]
gender_1_n = len(gender_1)
gender_1_avg = gender_1.mean()
gender_1_variance = (gender_1.std())**2
print(f"Gender 1 ---  Mean: {gender_1_avg}   N: {gender_1_n}   Variance: {gender_1_variance}")

Gender 1 ---  Mean: 814.0324088236293   N: 176526   Variance: 25206184.65721903


In [13]:
gender_2 = citibike[citibike["gender"] == 2]["tripduration"]
gender_2_avg = gender_2.mean()
gender_2_n = len(gender_2)
gender_2_variance = (gender_2.std())**2
print(f"Gender 2 ---- Mean: {gender_2_avg}    N: {gender_2_n}     Variance: {gender_2_variance}")

Gender 2 ---- Mean: 991.3610742785506    N: 41479     Variance: 50619713.486455254


In [14]:
se = math.sqrt((gender_1_variance/gender_1_n) + (gender_2_variance/gender_2_n))
print(f"Standard Deviation of Sampling Distribution: {se}")
difference_means = (gender_1_avg - gender_2_avg)
print(f"Difference of sample means {difference_means}")
z_score = difference_means / se
print(f"Z Score: {z_score}")
p_value = 2 * norm.cdf(difference_means, scale=se)
print(p_value)

Standard Deviation of Sampling Distribution: 36.92099509665831
Difference of sample means -177.3286654549213
Z Score: -4.802922158264667
1.5636668570619254e-06


In [15]:
# Probability is wayyyy too low, so we reject null hypothesis of equality.

## What numeric columns shouldn't be treated as numeric?

**Answer:**

## Dummify the `start station id` Variable

In [16]:
pd.get_dummies(citibike["start station id"])

,72,79,82,83,116,119,120,127,128,137,...,2006,2008,2009,2010,2012,2017,2021,2022,2023,3002
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224731,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
224732,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
224733,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
224734,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [17]:
dropped_citibike = citibike.loc[:,["tripduration", "birth year", "usertype","gender"]]
dropped_citibike.head()

,tripduration,birth year,usertype,gender
0,382,1991,Subscriber,1
1,372,1979,Subscriber,2
2,591,1948,Subscriber,2
3,583,1981,Subscriber,1
4,223,1990,Subscriber,1


In [18]:
modified_citibike = pd.concat([dropped_citibike, (pd.get_dummies(citibike["start station id"], drop_first = True)).astype(int)], axis=1)
modified_citibike[2006].describe()

count    224736.000000
mean          0.001909
std           0.043649
min           0.000000
25%           0.000000
50%           0.000000
75%           0.000000
max           1.000000
Name: 2006, dtype: float64

## Engineer a feature called `age` that shares how old the person would have been in 2014 (at the time the data was collected).

- Note: you will need to clean the data a bit.

In [19]:
modified_citibike[(modified_citibike["usertype"] == "Customer") & (modified_citibike["birth year"] == "\\N")] #we find that ALL customer usertypes dont have birth years or genders

,tripduration,birth year,usertype,gender,79,82,83,116,119,120,...,2006,2008,2009,2010,2012,2017,2021,2022,2023,3002
31,664,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55,836,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
222,1277,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
266,29906,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
293,2625,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
224385,280,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224438,1106,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224525,864,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
224536,683,\N,Customer,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [20]:
modified_citibike["age"] = modified_citibike["birth year"].apply(lambda x: np.nan if x == "\\N" else 2014 - int(x))

In [21]:
modified_citibike.head()
modified_citibike.drop(columns="birth year", inplace=True)

In [22]:
usertype = pd.get_dummies(modified_citibike["usertype"], drop_first=True)
encoded_usertype = pd.concat([modified_citibike, usertype], axis=1)
encoded_usertype.drop(columns = "usertype", inplace = True)

In [23]:
encoded_usertype.head()

,tripduration,gender,79,82,83,116,119,120,127,128,...,2009,2010,2012,2017,2021,2022,2023,3002,age,Subscriber
0,382,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,23.0,True
1,372,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,35.0,True
2,591,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,66.0,True
3,583,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,33.0,True
4,223,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,24.0,True


In [24]:
gender = pd.get_dummies(modified_citibike["gender"], drop_first=True)
final_df = pd.concat([encoded_usertype, gender], axis=1)
#final_df.drop(columns = "gender", inplace = True)

In [25]:
final_df.rename(columns={1:"gender_1", 2:"gender_2"}, inplace=True)
final_df.head()

,tripduration,gender,79,82,83,116,119,120,127,128,...,2012,2017,2021,2022,2023,3002,age,Subscriber,gender_1,gender_2
0,382,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,23.0,True,True,False
1,372,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,35.0,True,False,True
2,591,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,66.0,True,False,True
3,583,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,33.0,True,True,False
4,223,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,24.0,True,True,False


## Split your data into train/test data

Look at the size of your data. What is a good proportion for your split? **Justify your answer.**

Use the `tripduration` column as your `y` variable.

For your `X` variables, use `age`, `usertype`, `gender`, and the dummy variables you created from `start station id`. (Hint: You may find the Pandas `.drop()` method helpful here.)

**NOTE:** When doing your train/test split, please use random seed 123.

In [26]:
final_df.dropna(inplace=True)

In [27]:
X_train, X_test, Y_train, Y_split = train_test_split(final_df.loc[:,79:"gender_2"],final_df["tripduration"])

In [28]:
X_train.head()
X_train.columns = X_train.columns.astype(str)

## Fit a Linear Regression model in `sklearn` predicting `tripduration`.

In [29]:
citibike_regressor = LinearRegression()
citibike_regressor.fit(X_train, Y_train)

LinearRegression()

In [30]:
train_predictions = citibike_regressor.predict(X_train)

X_test.columns = X_test.columns.astype(str)
test_predictions = citibike_regressor.predict(X_test)

## Evaluate your model
Look at some evaluation metrics for **both** the training and test data. 
- How did your model do? Is it overfit, underfit, or neither?
- Does this model outperform the baseline? (e.g. setting $\hat{y}$ to be the mean of our training `y` values.)

In [31]:
zip(train_predictions, Y_train)

In [32]:
train_mse = sum([(x - y)**2 for x,y in zip(Y_train, train_predictions)]) / len(train_predictions)
print(train_mse)
#print(f"Training MSE: {mean_squared_error(Y_train, train_predictions)} ") #gives same thing

test_mse = sum([(x - y)**2 for x,y in zip(Y_split, test_predictions)]) / len(test_predictions)
print(test_mse)

mean_y_train = np.mean(Y_train)
baseline_mse = sum([(x - mean_y_train)**2 for x in Y_train])/len(Y_train)
print(baseline_mse)

32858576.82969436
21283809.242901403
32991340.95640852


In [33]:
print(f"R^2 on training set: {r2_score(Y_train, train_predictions)}")
print(f"R^2 on test set: {r2_score(Y_split, test_predictions)}")

R^2 on training set: 0.004024211288922053
R^2 on test set: -0.003918924354856745


## Fit a Linear Regression model in `statsmodels` predicting `tripduration`.

In [40]:
X_train = sm.add_constant(X_train)

In [44]:
model = sm.OLS(Y_train, X_train.astype(float))
results = model.fit()

In [46]:
print(results.summary())

                            OLS Regression Results                            
Dep. Variable:           tripduration   R-squared:                       0.004
Model:                            OLS   Adj. R-squared:                  0.002
Method:                 Least Squares   F-statistic:                     1.992
Date:                Wed, 09 Jul 2025   Prob (F-statistic):           6.25e-24
Time:                        20:35:55   Log-Likelihood:            -1.6470e+06
No. Observations:              163514   AIC:                         3.295e+06
Df Residuals:                  163182   BIC:                         3.298e+06
Df Model:                         331                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const        207.2612    809.643      0.256      0.7

## Using the `statsmodels` summary, test whether or not `age` has a significant effect when predicting `tripduration`.
- Be sure to specify your null and alternative hypotheses, and to state your conclusion carefully and correctly **in the context of your model**!

In [ ]:
#Null Hypothesis is that the Coefficient for Age is 0; alternative is that it is not 0.
# P_value is 0.001 * 2, which is 0.002; which is less than significance level of 0.01. This means that can reject the null hypothesis
# So age has a significant effect when predicting tripduration.

## Citi Bike is attempting to market to people who they think will ride their bike for a long time. Based on your modeling, what types of individuals should Citi Bike market toward?

In [ ]:
# Older people.